In [0]:
# -*- coding: utf-8 -*-
'''VGG16 model for Keras.

# Reference:

- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)

'''
from __future__ import print_function
import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
#from keras.applications.imagenet_utils import _obtain_input_shape
from keras.engine.topology import get_source_inputs
from google.colab import drive
drive.mount('/content/drive/')
root='/content/drive/My Drive/CNN_Basic/'

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
a=10
import pickle
f=open(root+'wts_voc_3','wb')
pickle.dump([a],f)
f.close()

In [0]:
def VGG16(include_top=True, weights='imagenet',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=1000):
    
    
    img_input = Input(shape=input_shape)
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(1024, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='sigmoid', name='predictions')(x)
    model = Model(img_input, x, name='vgg16')
    model.summary()
    # load weights
    #weights_path = root+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model.load_weights(weights_path)
    
    return model

In [0]:
classes=['person', 'bird', 'cat', 'cow', 'dog', 'horse', 'sheep','aeroplane', 'bicycle',
         'boat', 'bus', 'car', 'motorbike', 'train', 'bottle', 'chair','dining table',
         'potted plant', 'sofa', 'tvmonitor']
model = VGG16(include_top=True, weights='imagenet',input_shape=[224,224,3],classes=len(classes))
weights_path = root+'vgg16_weights_tf_dim_ordering_tf_kernels.h5'
#model.load_weights(weights_path)
    

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [0]:
import h5py
def load_wts(model):
  f = h5py.File(weights_path, 'r')
  print(list(f.keys()))
  #f['block1_conv1']['block1_conv1_W_1:0'].shape
  #model.get_layer('block1_conv1').get_weights()[1].shape

  layers=list(f.keys())
  for layer in layers:
    if layer[:5]=='block':
      l_type=layer.split('_')[1][:4]
    else:
      l_type=layer
    #print(l_type)
    if l_type =='conv':
      wts=f[layer][layer+'_W_1:0']
      bs=f[layer][layer+'_b_1:0']

      temp_layer=model.get_layer(layer)

      temp_layer.set_weights([wts,bs])
      #temp_layer.set_biases(bs)

      print('layer',layer,'type',l_type)
  return model
  #block1_conv1_W_1:0
  #block1_conv1_b_1:0
model=load_wts(model)

['block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool', 'fc1', 'fc2', 'flatten', 'predictions']
layer block1_conv1 type conv
layer block1_conv2 type conv
layer block2_conv1 type conv
layer block2_conv2 type conv
layer block3_conv1 type conv
layer block3_conv2 type conv
layer block3_conv3 type conv
layer block4_conv1 type conv
layer block4_conv2 type conv
layer block4_conv3 type conv
layer block5_conv1 type conv
layer block5_conv2 type conv
layer block5_conv3 type conv


In [0]:
from xml.etree import ElementTree as ET

def read_content(xml_file: str):
    objs=[] 
    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []
    filename = root.find('filename').text
    for boxes in root.iter('object'):

        
        obj = boxes.find('name').text
        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        list_with_all_boxes.append(list_with_single_boxes)
        objs.append(obj)
    return filename, list_with_all_boxes,objs

In [0]:
import glob
import cv2
import pandas as pd
save=False
if save==True:
  fnames=[]
  obj_list=[]
  i=0
  for file in glob.glob(root+'Annotations/*'):
    i+=1
    print(i,file)
    fname, boxes, objs = read_content(file)
    fnames.append(fname)
    obj_list.append(objs)
 
  df=pd.DataFrame([fnames,obj_list],index=['fname','objs']).transpose()
  df=df.sample(frac=1,random_state=10).reset_index(drop=True)
  n=len(df)
  split=int(0.8*n)
  train_data=df.loc[:split]
  val_data=df.loc[split:].reset_index(drop=True)
  import pickle
  #f=open(root+'_VOC_dfs','wb')
  #pickle.dump([train_data,val_data],f)
  #f.close()
import pickle
f=open(root+'_VOC_dfs','rb')
[train_data,val_data]=pickle.load(f)
f.close()

In [0]:
 def get_data(df):
  X=[]
  Y=[]
  i=0
  for i in range(len(df)):
    fname=df.loc[i,'fname']
    objs=df.loc[i,'objs']
    lbl=np.zeros(len(classes))
    for obj in classes:
      if obj in objs:
        ind=classes.index(obj)
        lbl[ind]=1
    Y.append(lbl)
    path=root+'VOC_Images/'+fname
    img=cv2.imread(path)
    img=cv2.resize(img,(224,224))
    X.append(img)
    #print(fname,objs,'\n',lbl)
    i+=1
    print(i)
    if i==10:
      #break
      pass
  return X,Y
#trainX,trainY=get_data(train_data)
#valX,valY=get_data(val_data)
save=False
if save==True:
  trainX,trainY=get_data(train_data)
  valX,valY=get_data(val_data)
  import pickle
  f=open(root+'_VOCtrXY','wb')
  pickle.dump([trainX,trainY],f)
  f.close()
  import pickle
  f=open(root+'_VOCvalXY','wb')
  pickle.dump([valX,valY],f)
  f.close()


In [0]:
import pickle
f=open(root+'_VOCtrXY','rb')
[trainX,trainY]=pickle.load(f)
f.close()

import pickle
f=open(root+'_VOCvalXY','rb')
[valX,valY]=pickle.load(f)
f.close()

trainX=np.array(trainX)
trainY=np.array(trainY)
valX=np.array(valX)
valY=np.array(valY)

In [0]:
train=False
if train==True:
  from keras import optimizers
  opt = optimizers.rmsprop(lr=0.000001, decay=1e-6)
  model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])

  history=model.fit(trainX,trainY,batch_size=100,epochs=20,validation_data=(valX,valY),shuffle=True)
  '''
  H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS)
'''

In [0]:
#model.save(root+'voc_trained_wts_20_class_30_epoch.h5')
model.load_weights(root+'_voc_trained_wts_20_class_30_epoch.h5')

In [0]:
from matplotlib import pyplot as plt
for i in range(20):
    #model = VGG16(include_top=True, weights='imagenet',input_shape=[224,224,3])

    #img_path = root+''
    #img = image.load_img(img_path, target_size=(224, 224))
    img=valX[i]
    plt.imshow(img)
    plt.show()
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)
    print('Input image shape:', x.shape)
    
    preds = model.predict(x)
    indx=np.argsort(-preds)
    for ind in indx[0][:3]:
      print(ind)
      pred=classes[ind]
      conf=preds[0][ind]
      print('Predicted:',pred,conf )


Output hidden; open in https://colab.research.google.com to view.

In [0]:
preds = model.predict(valX)
preds[preds>0.1]=1
preds[preds<0.01]=0

In [0]:
right=0
wrong=0
for j in range(len(classes)):
  right=0
  wrong=0
  for i in range(len(preds)):
    if preds[i][j]==valY[i][j]:
      right+=1
    else:
      wrong +=1
      
  print(classes[j])
  print((right/len(preds))*100,'%') #accuracy =((TP+TN)/ALL)

person
75.2741774675972 %
bird
95.0149551345962 %
cat
92.2233300099701 %
cow
95.21435692921236 %
dog
89.13260219341974 %
horse
93.8185443668993 %
sheep
97.70687936191426 %
aeroplane
97.80658025922233 %
bicycle
95.0149551345962 %
boat
95.91226321036889 %
bus
95.91226321036889 %
car
87.6370887337986 %
motorbike
95.71286141575274 %
train
96.01196410767697 %
bottle
91.32602193419741 %
chair
85.54336989032902 %
dining table
100.0 %
potted plant
99.90029910269193 %
sofa
90.32901296111665 %
tvmonitor
92.42273180458625 %


In [0]:
from sklearn.metrics.pairwise import cosine_similarity as cos_sm
csm_mean=0
for i in range(len(preds)):
  csm=cos_sm(valY[i:i+1],valY[i:i+1])
  csm_mean+=csm
csm_mean/=i+1
csm_mean

array([[0.97607178]])

In [0]:
from sklearn.metrics.pairwise import cosine_similarity as cos_sm
csm_mean=0
for i in range(len(preds)):
  csm=cos_sm(preds[i:i+1],valY[i:i+1])
  csm_mean+=csm
csm_mean/=i+1
csm_mean

array([[0.59632927]])

In [0]:
cos=0
for i in range(len(preds)):
  dot=np.dot(preds[i],valY[i])
  mag_a=np.linalg.norm(preds[i])
  mag_b=np.linalg.norm(valY[i])
  if mag_a==0 or mag_b==0:
    mag_a=1
    mag_b=1
  cos+=dot/(mag_a*mag_b)
  #print(cos)
cos/len(preds)

0.5963292778236671

In [0]:
np.mean(np.square(preds-valY))

0.05554454977161442

In [0]:
right=0
wrong=0
for i in range(len(preds)):
  if np.argmax(preds[i])==np.argmax(valY[i]):
    right+=1
  else:
    wrong+=1
right,wrong
#(1246, 134)
#(1256, 124)

(609, 394)